In [1]:
import progentrl as gentrl
import torch
import pandas as pd

from tqdm import tqdm_notebook

import warnings
warnings.simplefilter('ignore')

In [2]:
from pytorch_lightning import Trainer

# Reward Function
This section is for defining the reward function. 

In [3]:
from moses.metrics import mol_passes_filters, QED, SA, logP
from moses.metrics.utils import get_n_rings, get_mol


def get_num_rings_6(mol):
    r = mol.GetRingInfo()
    return len([x for x in r.AtomRings() if len(x) > 6])


def penalized_logP(mol_or_smiles, masked=False, default=-5):
    mol = get_mol(mol_or_smiles)
    if mol is None:
        return default
    reward = logP(mol) - SA(mol) - get_num_rings_6(mol)
    if masked and not mol_passes_filters(mol):
        return default
    return reward

In [4]:
! wget https://media.githubusercontent.com/media/molecularsets/moses/master/data/dataset_v1.csv

In [5]:
df = pd.read_csv('dataset_v1.csv')
df = df[df['SPLIT'] == 'train']
df['plogP'] = df['SMILES'].apply(penalized_logP)
df.to_csv('train_plogp_plogpm.csv', index=None)

In [6]:
md = gentrl.MolecularDataset(sources=[
    {'path':'train_plogp_plogpm.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'plogP' : 'plogP',
    }], 
    props=['plogP'])

from torch.utils.data import DataLoader
train_loader = DataLoader(md, batch_size=50, shuffle=True, num_workers=1, drop_last=True)

# Defining the Encoder, Decoder and Descriptors

In [7]:
enc = gentrl.RNNEncoder(latent_size=50)
dec = gentrl.DilConvDecoder(latent_input_size=50)
latent_descr = 50 * [('c', 20)]
feature_descr = [('c', 20)]

# Training the VAE (pretrain Step)

In [8]:
model = gentrl.gentrlVAE(enc,
                         dec,
                         train_loader,
                         latent_descr,
                         feature_descr, load_model='./model/')

## Pytorch-Lightning trainer (For more details refer [this](https://pytorch-lightning.readthedocs.io/en/latest/trainer.html) )
- max_epochs: maximum Epochs
- gpus: No of gpus (-1 means all of them and 0 means cpu)

In [9]:
trainer = Trainer(max_epochs=20, gpus=-1)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


In [10]:
trainer.fit(model)  # Training the model


   | Name                 | Type           | Params
----------------------------------------------------
0  | enc                  | RNNEncoder     | 888 K 
1  | enc.embs             | Embedding      | 7 K   
2  | enc.rnn              | GRU            | 789 K 
3  | enc.final_mlp        | Sequential     | 91 K  
4  | enc.final_mlp.0      | Linear         | 65 K  
5  | enc.final_mlp.1      | LeakyReLU      | 0     
6  | enc.final_mlp.2      | Linear         | 25 K  
7  | dec                  | DilConvDecoder | 596 K 
8  | dec.latent_fc        | Linear         | 6 K   
9  | dec.input_embeddings | Embedding      | 3 K   
10 | dec.logits_1x1_layer | Conv1d         | 3 K   
11 | dec.parameters       | ParameterList  | 596 K 
12 | lp                   | LP             | 1 M   
13 | lp.parameters        | ParameterList  | 1 M   


1

## Saving the model

In [11]:
gentrl.save(model, './model')  # Returns true on successful completion

True

# Reinforcement Learning (train_rl Step)

In [12]:
model = gentrl.gentrlRL(penalized_logP, 
                        enc,
                        dec,
                        latent_descr,
                        feature_descr,
                       load_model='./model/')

In [13]:
trainer = Trainer(max_epochs=10, gpus=-1)

GPU available: True, used: True
No environment variable for node rank defined. Set as 0.
CUDA_VISIBLE_DEVICES: [0]


In [14]:
trainer.fit(model)


   | Name                 | Type           | Params
----------------------------------------------------
0  | enc                  | RNNEncoder     | 888 K 
1  | enc.embs             | Embedding      | 7 K   
2  | enc.rnn              | GRU            | 789 K 
3  | enc.final_mlp        | Sequential     | 91 K  
4  | enc.final_mlp.0      | Linear         | 65 K  
5  | enc.final_mlp.1      | LeakyReLU      | 0     
6  | enc.final_mlp.2      | Linear         | 25 K  
7  | dec                  | DilConvDecoder | 596 K 
8  | dec.latent_fc        | Linear         | 6 K   
9  | dec.input_embeddings | Embedding      | 3 K   
10 | dec.logits_1x1_layer | Conv1d         | 3 K   
11 | dec.parameters       | ParameterList  | 596 K 
12 | lp                   | LP             | 1 M   
13 | lp.parameters        | ParameterList  | 1 M   


1

In [15]:
gentrl.save(model, './model')

True

# Sampling Molecules (Sampling step)

In [16]:
model(10)

['O=CCC(=O)(CO)C(N)=C(C)O)Sc1Cl',
 'Nc1cc(C)c(C)c1cc(N)c2c2C(=C(CO)N1',
 'CC1C(=O)C(C)CC(C#N)N(CO)c2ccccc1Cl',
 'CC(Cl)c(C21nc1)=C(C1)c(N=O)(S2C)[c2c=O)C1',
 'CC1nc2c1ccnc(N)c2ccc(O)c1',
 'CC1c(O)Nc1cn1',
 'Cc1nccc(OC(C(CO)C(=O)CC1=C)O=C(C',
 'CNCc2c1nc2c2c(NS(C2)nnc2c(c2c1nc2#',
 'CC1nc2cc(Cl)ccc2cncnc1N(CC)c2cc(Cl)Cl',
 'CCC1(CCC(oC(C)=O)N1']